# Women Cloth Reviews Prediction with Multi Nomial Naïve Bayes
---
Multinomial Naïve Bayes algorithm is a probabilistic learning method that is mostly used in Natural Language Processing. It is also suitable for text classification with discrete feature. In this project it will be used to build a women cloth review prediction model.

Data Source: [YBIFoundation/ProjectHub-MachineLearning Women Clothing Commerce Review dataset](https://raw.githubusercontent.com/YBIFoundation/ProjectHub-MachineLearning/main/Women%20Clothing%20E-Commerce%20Review.csv)

### Importing Libraries

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Importing Dataset

In [48]:
df = pd.read_csv("https://raw.githubusercontent.com/YBIFoundation/ProjectHub-MachineLearning/main/Women%20Clothing%20E-Commerce%20Review.csv")

### Data visualization

Samples of the data is visualized to better understand how it is structured.

In [49]:
df.head()

,Clothing ID,Age,Title,Review,Rating,Recommended,Positive Feedback,Division,Department,Category
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Clothing ID        23486 non-null  int64 
 1   Age                23486 non-null  int64 
 2   Title              19676 non-null  object
 3   Review             22641 non-null  object
 4   Rating             23486 non-null  int64 
 5   Recommended        23486 non-null  int64 
 6   Positive Feedback  23486 non-null  int64 
 7   Division           23472 non-null  object
 8   Department         23472 non-null  object
 9   Category           23472 non-null  object
dtypes: int64(5), object(5)
memory usage: 1.8+ MB


In [51]:
df.shape

(23486, 10)

### Data preprocessing

Calling `isna()` method along with the `sum()` method on dataframe `df` to find the Review columns with no review text for further processing.

In [52]:
df.isna().sum()

,0
Clothing ID,0
Age,0
Title,3810
Review,845
Rating,0
Recommended,0
Positive Feedback,0
Division,14
Department,14
Category,14


Filling missing values in the Review column with the value `No review is given`.

In [53]:
import numpy as np

df.loc[df['Review'] == "", 'Review'] = np.nan
df['Review'] = df['Review'].fillna("No review is given")

df.isna().sum()


,0
Clothing ID,0
Age,0
Title,3810
Review,0
Rating,0
Recommended,0
Positive Feedback,0
Division,14
Department,14
Category,14


In [54]:
df['Review']

,Review
0,Absolutely wonderful - silky and sexy and comf...
1,Love this dress! it's sooo pretty. i happene...
2,I had such high hopes for this dress and reall...
3,"I love, love, love this jumpsuit. it's fun, fl..."
4,This shirt is very flattering to all due to th...
...,...
23481,I was very happy to snag this dress at such a ...
23482,"It reminds me of maternity clothes. soft, stre..."
23483,"This fit well, but the top was very see throug..."
23484,I bought this dress for a wedding i have this ...


### Defining Target Variable (y) and Feature Variables (X)

In [55]:
df.columns

Index(['Clothing ID', 'Age', 'Title', 'Review', 'Rating', 'Recommended',
       'Positive Feedback', 'Division', 'Department', 'Category'],
      dtype='object')

In [56]:
x = df['Review']
y = df['Rating']
df['Rating'].value_counts()

,count
Rating,
5,13131
4,5077
3,2871
2,1565
1,842


### Train Test Split

In [57]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, stratify=y, random_state=2529)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((16440,), (7046,), (16440,), (7046,))

### Getting Tokens from Feature Text

Using `CountVectorizer` from sci-kit learn

In [58]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(lowercase=True, analyzer='word', ngram_range=(2, 3), stop_words='english', max_features=50000)
x_train = cv.fit_transform(x_train)
cv.get_feature_names_out()

array(['00 big', '00 dress', '00 fit', ..., 'zipper zip',
       'zippered pockets', 'zippers buttons'], dtype=object)

In [59]:
x_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [60]:
x_test = cv.fit_transform(x_test)
cv.get_feature_names_out()

array(['00 24 xs', '00 aren', '00 aren size', ..., 'zone roomy looking',
       'zone style', 'zone style daughter'], dtype=object)

In [61]:
x_test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Model training

Using Multinomial Naïve Bayes algorithm, which is implemented in sci-kit as `MultinomialNB`

In [62]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

### Model prediction

In [63]:
y_pred = model.predict(x_test)
y_pred.shape

(7046,)

In [64]:
y_pred

array([1, 1, 5, ..., 5, 2, 4])

Getting probability of each predicted class

In [65]:
model.predict_proba(x_test)

array([[9.07837369e-01, 8.41329696e-02, 7.24092571e-03, 4.73018576e-04,
        3.15717214e-04],
       [6.45225066e-01, 9.55649861e-02, 6.10725686e-02, 3.63722167e-02,
        1.61765162e-01],
       [4.68138388e-02, 7.46791973e-02, 1.10754401e-01, 1.46336529e-01,
        6.21416034e-01],
       ...,
       [2.05931939e-01, 1.01880401e-01, 7.80261733e-02, 2.40206103e-01,
        3.73955385e-01],
       [1.02387565e-01, 8.58093177e-01, 3.76665293e-02, 1.71993856e-03,
        1.32789420e-04],
       [7.77751684e-02, 7.10210302e-02, 4.83544487e-02, 7.76922612e-01,
        2.59267404e-02]])

### Model evaluation

In [66]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))

[[  63   46   35   47   62]
 [ 148   80   72   70  100]
 [ 286  178  126  113  158]
 [ 562  302  187  201  271]
 [1303  821  493  536  786]]


In [67]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.03      0.25      0.05       253
           2       0.06      0.17      0.08       470
           3       0.14      0.15      0.14       861
           4       0.21      0.13      0.16      1523
           5       0.57      0.20      0.30      3939

    accuracy                           0.18      7046
   macro avg       0.20      0.18      0.15      7046
weighted avg       0.39      0.18      0.22      7046



### Recategorizing ratings as Poor (0) and Good (1)

In [68]:
df["Rating"].value_counts()

,count
Rating,
5,13131
4,5077
3,2871
2,1565
1,842


re-rating 1,2,3 as 0 and 4,5 as 1

In [69]:
df.replace({'Rating': { 1:0, 2:0, 3:0, 4:1, 5:1 }}, inplace=True)
y = df['Rating']
x = df['Review']

### Train Test Split

In [70]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, stratify=y, random_state=2529)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((16440,), (7046,), (16440,), (7046,))

### Getting Tokens from Feature Text

In [71]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(lowercase=True, analyzer='word', ngram_range=(2, 3), stop_words='english', max_features=50000)
x_train = cv.fit_transform(x_train)
x_test = cv.fit_transform(x_test)

### Model re-training

In [72]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

### Model prediction

In [73]:
y_pred = model.predict(x_test)
y_pred.shape

(7046,)

In [74]:
y_pred

array([0, 0, 1, ..., 0, 1, 1])

### Model evaluation

In [75]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))

[[ 759  824]
 [2809 2654]]


In [76]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.21      0.48      0.29      1583
           1       0.76      0.49      0.59      5463

    accuracy                           0.48      7046
   macro avg       0.49      0.48      0.44      7046
weighted avg       0.64      0.48      0.53      7046



### Explanation

This project is focused on building a prediction model. At first, the all required libraries and a test dataset are imported. The dataset was evaluated and pre processed to prepare for it for processing, then a portion of it was kept for testing and the rest was used to train the model. The model was used to get some prediction dataset. Finnaly, prediction accuracy was checked against the test dataset, some adjusment were made and the model was re-trained for better accuracy.